# Bulk CSV Upload
*A notebook to read csvs and upload their data as [hosted feature layers](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm) in ArcGIS*

In [1]:
# common imports
import os
import pandas as pd
from arcgis.gis import *
from arcgis.features import SpatialDataFrame
from arcgis.geometry import Point

## User Input

* **gis**: your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/)
* **dir_path**: path to directory with the CSVs
* **coord_fields**: map Point fields to CSV columns

In [ ]:
gis = GIS("https://www.arcgis.com", "<USERNAME>", "<PASSWORD>")

dir_path = "<DIRECTORY PATH>"
coord_fields = {
    "x": "<X COLUMN>",
    "y": "<Y COLUMN>",
    #"z": "<Z COLUMN (optional)>"
}

## Execution

In [ ]:
# create a new folder in the GIS to store the layers
folder = os.path.basename(dir_path)
gis_folder = gis.content.create_folder(folder)

In [3]:
# get an array of all the csvs in the directory
csvs = [file for file in os.listdir(dir_path) if file.endswith('.csv')]
csv_paths = [os.path.join(dir_path, csv) for csv in csvs]
n_paths = len(csv_paths)
print("Pushing {0} csvs".format(n_paths))

In [4]:
# construct a point from a dataframe row
def get_point_for_row(row, coord_fields):
    p_dict = {}
    p_dict['x'] = row[coord_fields['x']]
    p_dict['y'] = row[coord_fields['y']]
    if 'z' in coord_fields and coord_fields['z']:
        p_dict['z'] = row[coord_fields['z']]
    return Point(p_dict)

In [5]:
# loop through csvs, build spatial dataframe, upload layers to GIS, move to new folder
for i, csv_path in enumerate(csv_paths):
    df = pd.read_csv(csv_path)
    df_geom = df.apply(lambda row: get_point_for_row(row, coord_fields), axis=1)
    sdf = SpatialDataFrame(data=df, geometry=df_geom)
    
    title = os.path.splitext(os.path.basename(csv_path))[0]
    csv_lyr = gis.content.import_data(sdf, title=title)
    csv_lyr.move(gis_folder)
    
    print("{0}/{1}".format(i + 1, n_paths))